# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,qa'en,33.7265,59.1844,6.51,49,29,2.23,IR,1702260736
1,1,port-aux-francais,-49.3500,70.2167,4.12,79,75,9.51,TF,1702260736
2,2,novogornyy,55.6300,60.7919,-30.94,96,37,3.01,RU,1702260737
3,3,krasnokamensk,50.0979,118.0369,-29.25,96,13,1.13,RU,1702260737
4,4,yellowknife,62.4560,-114.3525,-11.71,92,100,10.29,CA,1702260575


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
# YOUR CODE HERE

# Display the map
# YOUR CODE HERE

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

In [3]:
%%capture --no-display

city_plot = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    scale = 1,
    color = "City",
    alpha = 0.5
)

city_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
vacation_cities = city_data_df.loc[(city_data_df['Max Temp'] < 27) & (city_data_df['Max Temp'] > 21) & (city_data_df['Wind Speed'] < 4.5) & (city_data_df['Cloudiness'] == 0)]

# Drop any rows with null values
vacation_cities = vacation_cities.dropna(how='any')

# Display sample data
vacation_cities

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
17,17,adamstown,-25.0660,-130.1015,21.29,56,0,2.60,PN,1702260746
28,28,margaret river,-33.9500,115.0667,24.32,29,0,2.54,AU,1702260751
105,105,lagos,6.5833,3.7500,25.52,85,0,1.27,NG,1702260542
120,120,tazacorte,28.6290,-17.9293,21.19,41,0,4.47,ES,1702260731
184,184,cabo san lucas,22.8909,-109.9124,25.98,58,0,2.06,MX,1702260835
196,196,maceio,-9.6658,-35.7353,23.69,100,0,1.54,BR,1702260732
275,275,barra de santo antonio,-9.4047,-35.5072,23.60,81,0,3.55,BR,1702260694
306,306,zhanjiang,21.2815,110.3427,26.63,67,0,4.11,CN,1702260895
364,364,belmonte,-15.8631,-38.8828,24.61,86,0,2.76,BR,1702260944
377,377,prado,-17.3411,-39.2208,23.53,88,0,0.24,BR,1702260950


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
vacation_cities.copy()
hotel_df = pd.DataFrame(vacation_cities)

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ""
hotel_df

# Display sample data
# YOUR CODE HERE

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
17,17,adamstown,-25.0660,-130.1015,21.29,56,0,2.60,PN,1702260746,
28,28,margaret river,-33.9500,115.0667,24.32,29,0,2.54,AU,1702260751,
105,105,lagos,6.5833,3.7500,25.52,85,0,1.27,NG,1702260542,
120,120,tazacorte,28.6290,-17.9293,21.19,41,0,4.47,ES,1702260731,
184,184,cabo san lucas,22.8909,-109.9124,25.98,58,0,2.06,MX,1702260835,
196,196,maceio,-9.6658,-35.7353,23.69,100,0,1.54,BR,1702260732,
275,275,barra de santo antonio,-9.4047,-35.5072,23.60,81,0,3.55,BR,1702260694,
306,306,zhanjiang,21.2815,110.3427,26.63,67,0,4.11,CN,1702260895,
364,364,belmonte,-15.8631,-38.8828,24.61,86,0,2.76,BR,1702260944,
377,377,prado,-17.3411,-39.2208,23.53,88,0,0.24,BR,1702260950,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories":'accommodation.hotel',
    "apiKey":geoapify_key,
    'limit':20   
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row['Lat']
    longitude = row['Lng']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
adamstown - nearest hotel: No hotel found
margaret river - nearest hotel: Margaret River Hotel
lagos - nearest hotel: No hotel found
tazacorte - nearest hotel: App Leyma
cabo san lucas - nearest hotel: Comfort Rooms
maceio - nearest hotel: Maceió Hostel e Pousada
barra de santo antonio - nearest hotel: Resort Fazenda Fiore
zhanjiang - nearest hotel: 中英国际酒店
belmonte - nearest hotel: No hotel found
prado - nearest hotel: Pousada Recanto do Prado
perez - nearest hotel: Del Paseo
musaffah - nearest hotel: No hotel found


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
17,17,adamstown,-25.0660,-130.1015,21.29,56,0,2.60,PN,1702260746,No hotel found
28,28,margaret river,-33.9500,115.0667,24.32,29,0,2.54,AU,1702260751,Margaret River Hotel
105,105,lagos,6.5833,3.7500,25.52,85,0,1.27,NG,1702260542,No hotel found
120,120,tazacorte,28.6290,-17.9293,21.19,41,0,4.47,ES,1702260731,App Leyma
184,184,cabo san lucas,22.8909,-109.9124,25.98,58,0,2.06,MX,1702260835,Comfort Rooms
196,196,maceio,-9.6658,-35.7353,23.69,100,0,1.54,BR,1702260732,Maceió Hostel e Pousada
275,275,barra de santo antonio,-9.4047,-35.5072,23.60,81,0,3.55,BR,1702260694,Resort Fazenda Fiore
306,306,zhanjiang,21.2815,110.3427,26.63,67,0,4.11,CN,1702260895,中英国际酒店
364,364,belmonte,-15.8631,-38.8828,24.61,86,0,2.76,BR,1702260944,No hotel found
377,377,prado,-17.3411,-39.2208,23.53,88,0,0.24,BR,1702260950,Pousada Recanto do Prado


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [10]:
%%capture --no-display

# Configure the map plot
city_plot2 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    scale = 1,
    color = "City",
    hover_cols = ['Hotel Name', 'Country'],
    alpha = 0.5
)


# Display the map
city_plot2

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)